In [21]:
import json
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from collections import Counter, OrderedDict
import random
import matplotlib
from torch.utils.data import DataLoader, TensorDataset
import obspy
from scipy import signal
import os

In [33]:
#读取噪声数据
def noise_data_read(hdf5_path,json_path,num):
    # load the json file
    noise_json_file = json.load(open(json_path, 'r'))
    # 随机取num个波形
    noise_key_list_1 = noise_json_file['Earthquake']['Junior_A']
    noise_key_list_2 = noise_json_file['Earthquake']['Expert_A']
    noise_key_list_3 = noise_json_file['Earthquake']['Expert_B']
    noise_key_list_4 = noise_json_file['Earthquake']['Expert_C']
    # 合并四个列表
    noise_key_list = noise_key_list_1 + noise_key_list_2 + noise_key_list_3 + noise_key_list_4
    print(len(noise_key_list))
    # # 将合并后的列表转换为 PyTorch 张量
    # noise_key_list = torch.tensor(noise_key_list)

    noise_dataset=[]
    noise_list=[]
    hdf5_file = h5py.File(hdf5_path, 'r')
    for key in noise_key_list :
        waveform = hdf5_file.get(key)[()]
        if len(waveform)>2000 and waveform.shape[1] == 3:
            # 切片操作
            if waveform.shape[0] <3000 :
                # 获取需要填充的零的数量
                padding_size = 3000 - waveform.shape[0]
                # 计算每列的均值
                column_means = np.mean(waveform, axis=0)
                # 创建一个填充后的数组
                padded_waveform = np.zeros((3000, 3))
                # 在每列下面填充相应列的均值
                for k in range(3):
                    padded_waveform[:waveform.shape[0], k] = waveform[:, k]
                    padded_waveform[waveform.shape[0]:, k] = column_means[k]

            else:
                padded_waveform=waveform[:3000,:]
            data = np.array(padded_waveform)
            noise_dataset.append(data)
            noise_list.append(key)

    print(f'noise_train_dataset.shape:{len(noise_list)}')
    noise_dataset=torch.tensor(noise_dataset)
    print(f'noise_train_dataset.shape:{noise_dataset.shape}')
    # print(f'noise_test_dataset.shape:{noise_list.shape}')

    return noise_dataset,noise_list

In [34]:
noise_hdf5_path='/home/zzy/Python projects/Dataset/DiTing2.0/CENC_DiTingv2_noise_set.hdf5'
noise_json_path='/home/zzy/Python projects/Dataset/DiTing2.0/CENC_DiTingv2_noise_set.json'

noise_train_dataset,noise_test_dataset=noise_data_read(noise_hdf5_path,noise_json_path,6000)


37032
306
1570
52308
91216
noise_train_dataset.shape:579
noise_train_dataset.shape:torch.Size([579, 3000, 3])


ValueError: not enough values to unpack (expected 4, got 2)